In [64]:
#!/Users/CN/Documents/Conferences/NCM21/ncm_highlights_article/ncm_highlights/venv python
# ------------------------------------------------------------------------------
# Script name:  ncm_world_map.py
#
# Description:
#               Notebook that visualises world map from NCM attendence data
#
# Author:       Caroline Nettekoven, 2021
#
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2, country_name_to_country_alpha3
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster

# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------

In [57]:
# ------------------------------ Define helper functions --------------------------------
# function to convert to alpah2 country codes and continents
def get_continent(col):
    try:
        cn_a2_code = country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown'
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown'
    return (cn_a2_code, cn_continent)

def get_alpha3(col):
    try:
        cn_a3_code = country_name_to_country_alpha3(col)
    except:
        cn_a3_code = 'Unknown'
    return (cn_a3_code)


# function to get longitude and latitude data from country name
geolocator = Nominatim(user_agent='ncm_highlights')
def geolocate(country_code, country_name):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country_code)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        try:
            loc = geolocator.geocode(country_name)
        except:
            # Return missing value
            return np.nan


In [58]:

# ------------------------------------------------------------------------------
# --------------------------------- Get Data -----------------------
file = '/Users/CN/Documents/Conferences/NCM21/ncm_highlights_article/NCMAnonymizedDelegateRegistration.csv'
data = pd.read_csv(file, delimiter=';')
d = {'CountryName': data.COUNTRY.value_counts(
).index, 'Members': data.COUNTRY.value_counts().values ,'Percentage': data.COUNTRY.value_counts().values * 100 / len(data)}
df = pd.DataFrame(data=d)

# Replace Chinas name
df.at[df['CountryName'] == "China, People's Republic of", 'CountryName'] = "China"


In [59]:

# ------------------------------ Get Country and Continent ------------------------------------

df['Country'] = None
df['Continent'] = None
for i in range(len(df)):
    country = get_continent(df.CountryName[i])[0]
    continent = get_continent(df.CountryName[i])[1]
    country_code_alpha3 = get_alpha3(df.CountryName[i])
    df.at[i, 'Country'] = country
    df.at[i, 'Continent'] = continent
    df.at[i, 'Country_A3'] = country_code_alpha3



In [15]:
# ------------------------------ Get Coordinates (Latitude and Longitude) ------------------------------------
df['Latitude'] = None
df['Longitude'] = None
for i in range(len(df)):
    coordinates = geolocate(df.CountryName[i], df.Country[i])
    Latitude = coordinates[0]
    Longitude = coordinates[1]
    df.at[i, 'Latitude'] = Latitude
    df.at[i, 'Longitude'] = Longitude



In [18]:
# ------------------------------ Draw world map ------------------------------------

# Create a world map to show distributions of users
# empty map
world_map = folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)


# for each coordinate, create circlemarker of user percent
for i in range(len(df)):
    lat = df.at[i, 'Latitude']
    long = df.at[i, 'Longitude']
    radius = 5
    popup_text = """Country : {}<br>
                    Members : {}<br>"""
    popup_text = popup_text.format(df.at[i, 'Country'],
                                   df.at[i, 'Members']
                                   )
    folium.CircleMarker(location=[
        lat, long], radius=radius, popup=popup_text, fill=True).add_to(marker_cluster)

# show the map
world_map

In [60]:
df.head()

,CountryName,Members,Percentage,Country,Continent,Country_A3
0,United States,312,44.067797,US,NA,USA
1,Canada,90,12.711864,CA,NA,CAN
2,United Kingdom,47,6.638418,GB,EU,GBR
3,Japan,35,4.943503,JP,AS,JPN
4,Germany,30,4.237288,DE,EU,DEU
